In [1]:
#import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [3]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
import torch.nn.functional as F

In [4]:
h4ca20k_py = load_dataset("graycatHCO3/CodeAlpaca-20K-Python")
#h4ca20k_py = load_dataset("./data/codeNLU/h4ca20k_py/h4ca20k_py_.parquet")

In [5]:

print(h4ca20k_py)




DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 4777
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 548
    })
})


# 1. bert-base-uncased

## 1) Training

In [6]:
from transformers import BertTokenizer, EncoderDecoderModel


In [7]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenizer_fn(examples):
    inputs = bert_tokenizer(examples["completion"], max_length=128, padding="max_length", truncation=True)
    with bert_tokenizer.as_target_tokenizer():
        labels = bert_tokenizer(examples["prompt"], max_length=128, padding="max_length", truncation=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

bert_tkd = h4ca20k_py.map(tokenizer_fn, batched=True)

print(bert_tkd["train"][0])

{'prompt': 'Write a reusuable function in Python that takes two string variables and returns the longest string.\n', 'completion': 'def longest_string(str1, str2):\n    if len(str1) > len(str2):\n        return str1\n    else:\n        return str2', 'input_ids': [101, 13366, 6493, 1035, 5164, 1006, 2358, 2099, 2487, 1010, 2358, 2099, 2475, 1007, 1024, 2065, 18798, 1006, 2358, 2099, 2487, 1007, 1028, 18798, 1006, 2358, 2099, 2475, 1007, 1024, 2709, 2358, 2099, 2487, 2842, 1024, 2709, 2358, 2099, 2475, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [8]:
# 将数据转换为 PyTorch Tensor
def collate_fn(batch):
    input_ids = torch.tensor([item['input_ids'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch])
    return {"input_ids": input_ids, "labels": labels}

In [9]:
h4ca_train = bert_tkd["train"].shuffle(seed=42)
h4ca_test = bert_tkd["test"].shuffle(seed=42)

In [10]:
train_loader = DataLoader(h4ca_train, batch_size=16, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(h4ca_test, batch_size=16, shuffle=False, collate_fn=collate_fn)

In [11]:
bert_ED = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

bert_ED.config.decoder_start_token_id = bert_tokenizer.cls_token_id

bert_ED.config.pad_token_id = bert_tokenizer.pad_token_id

bert_ED.config.vocab_size = bert_ED.config.decoder.vocab_size

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.e

In [12]:
bert_ED.config.max_length = 128
bert_ED.config.min_length = 10
bert_ED.config.early_stopping = True

bert_ED.config.no_repeat_ngram_size = 2

bert_ED.config.num_beams = 4
bert_ED.config.length_penalty = 2.0

In [15]:
bert_ED = bert_ED.to(device)

In [16]:
optimizer = AdamW(bert_ED.parameters(), lr=5e-5)

In [17]:
bert_ED.train()

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [24]:
for epoch in range(3):
    for batch in train_loader:
        inputs, labels = batch["input_ids"].to(device), batch["labels"].to(device)
        outputs = bert_ED(input_ids=inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

d:\Disk_D\programming_software\AI\Anaconda3\envs\pettingZoo_Langchain\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
d:\Disk_D\programming_software\AI\Anaconda3\envs\pettingZoo_Langchain\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.

Epoch: 1, Loss: 16.118013381958008
Epoch: 1, Loss: 11.00251293182373
Epoch: 1, Loss: 8.692075729370117
Epoch: 1, Loss: 8.03219223022461
Epoch: 1, Loss: 7.369291305541992
Epoch: 1, Loss: 7.649518966674805
Epoch: 1, Loss: 7.242735862731934
Epoch: 1, Loss: 6.696038246154785
Epoch: 1, Loss: 6.145937919616699
Epoch: 1, Loss: 5.211989879608154
Epoch: 1, Loss: 4.988256931304932
Epoch: 1, Loss: 4.535675525665283
Epoch: 1, Loss: 4.125432014465332
Epoch: 1, Loss: 3.6398065090179443
Epoch: 1, Loss: 3.783958673477173
Epoch: 1, Loss: 3.3075673580169678
Epoch: 1, Loss: 3.2059075832366943
Epoch: 1, Loss: 3.3308868408203125
Epoch: 1, Loss: 2.7038848400115967
Epoch: 1, Loss: 2.354180335998535
Epoch: 1, Loss: 2.6275010108947754
Epoch: 1, Loss: 2.2951841354370117
Epoch: 1, Loss: 2.351940155029297
Epoch: 1, Loss: 2.2270290851593018
Epoch: 1, Loss: 2.2768399715423584
Epoch: 1, Loss: 1.9741548299789429
Epoch: 1, Loss: 1.7948312759399414
Epoch: 1, Loss: 2.753190040588379
Epoch: 1, Loss: 1.9613730907440186
Ep

In [18]:
import transformers
transformers.logging.set_verbosity_error()


In [25]:

torch.save(bert_ED, './parameters/weights/transformers/bert_ED/bert_ED_full.pth')
torch.save(bert_ED.state_dict(), './parameters/weights/transformers/bert_ED/bert_ED_state_dict.pth')


## 2) Testing

In [5]:
import torch
from transformers import BertTokenizer, EncoderDecoderModel
from torch.utils.data import DataLoader

In [12]:
bert_ED = torch.load('./parameters/weights/transformers/bert_ED/bert_ED_full.pth')

In [6]:
bert_tk = BertTokenizer.from_pretrained('bert-base-uncased')

In [14]:
bert_ED.config.decoder_start_token_id = bert_tk.cls_token_id

bert_ED.config.pad_token_id = bert_tk.pad_token_id

bert_ED.config.vocab_size = bert_ED.config.decoder.vocab_size

In [15]:
bert_ED.to(device)
bert_ED.eval()

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [16]:
bert_ED.config.decoder_start_token_id = bert_tk.cls_token_id

In [17]:
test_loader = DataLoader(h4ca_test, batch_size=16, shuffle=False, collate_fn=collate_fn)

In [18]:
total_loss = 0.0

In [19]:
for batch in test_loader:
    inputs, labels = batch["input_ids"].to(device), batch["labels"].to(device)
    with torch.no_grad():
        outputs = bert_ED(input_ids=inputs, labels=labels, attention_mask=batch.get("attention_mask", None))
        loss = outputs.loss
        total_loss += loss.item()

average_loss = total_loss / len(test_loader)
print(f"Average Loss: {average_loss}")

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
d:\Disk_D\programming_software\AI\Anaconda3\envs\pettingZoo_Langchain\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
d:\Disk_D\programming_software\AI\Anaconda3\envs\pettingZoo_Langchain\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. 

Average Loss: 0.48188516497612


In [19]:
bert_ED.generation_config.bos_token_id = bert_tk.bos_token_id
bert_ED.generation_config.pad_token_id = bert_tk.pad_token_id
bert_ED.generation_config.eos_token_id = bert_tk.eos_token_id
bert_ED.generation_config.decoder_start_token_id = bert_tk.cls_token_id

In [20]:
for i, batch in enumerate(test_loader):
    if i > 2: 
        break
    inputs = batch["input_ids"].to(device)
    with torch.no_grad():
        generated_ids = bert_ED.generate(input_ids=inputs, decoder_start_token_id=bert_tk.bos_token_id, num_beams=5, max_length=50, early_stopping=True)
        generated_text = bert_tk.batch_decode(generated_ids, skip_special_tokens=True)
    print(f"Batch {i+1} generated texts: {generated_text}")

Batch 1 generated texts: ['write a python code to print all prime numbers in a given list. list = [ 1, 2, 3, 4, 5, 6 ]', 'write a python script that takes a string as input and prints out all the characters in the string. string = " hello world " " "', 'write a python script that takes a string as input and prints out all the characters in the string. string = " hello world! "', 'write a python script to print all prime numbers between 0 and 10.', 'write a python script to print all prime numbers between 0 and 10.', 'create a function that takes in a list of strings and returns the longest substring of each string. string = " this is a string " "', 'create a python program to calculate the factorial of a given number.', 'create a function in python that takes a list of integers as an argument and returns a new list of integers. [ 1, 2, 3, 4, 5, 6 ]', 'create a web scraper using python.', 'write a python script to convert a given text to a dictionary.', 'write a python script to convert

**Training utilized about 16GB, Interfence utilized about 7GB**

In [22]:
print(sum(p.numel() for p in bert_ED.parameters())/1e9,'B parameters in bert_ED')

0.247363386 B parameters in bert_ED


## 3) Extracting Encoder

In [23]:
bert_EN = bert_ED.get_encoder()

In [24]:
bert_EN.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

**[batch_size, sequence_length, 768]**

cls_output = outputs.last_hidden_state[:, 0, :] \
[batch_size, 768]

In [25]:
print(sum(p.numel() for p in bert_EN.parameters())/1e9,'B parameters in bert_EN')

0.10948224 B parameters in bert_EN


In [28]:
from datetime import datetime
cur_time = datetime.now().strftime('%Y-%m-%d_%H') 

torch.save(bert_EN, './parameters/weights/encoders/bert_EN/bert_EN_full_{curtime}.pth')
torch.save(bert_EN.state_dict(), './parameters/weights/encoders/bert_EN/bert_EN_state_dict_{curtime}.pth')

In [8]:
from src.utils.data_utils import find_latest_file

encoders_dir = './parameters/weights/encoders/bert_EN'
spec_en = ''
encoder = torch.load(spec_en) if spec_en else torch.load(find_latest_file(encoders_dir))

In [9]:
encoder.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          